In [ ]:
!wget -N "https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/CNN_model_dataset.zip"
!unzip -qo CNN_model_dataset.zip
!rm CNN_model_dataset.zip

--2024-01-12 09:54:55--  https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/CNN_model_dataset.zip
Resolving cainvas-static.s3.amazonaws.com (cainvas-static.s3.amazonaws.com)... 16.12.36.111, 52.219.66.40, 16.12.40.79, ...
Connecting to cainvas-static.s3.amazonaws.com (cainvas-static.s3.amazonaws.com)|16.12.36.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170317173 (162M) [application/x-zip-compressed]
Saving to: ‘CNN_model_dataset.zip’

CNN_model_dataset.z  44%[=======>            ]  72.57M  11.8MB/s    eta 10s    

Importer les libraries

In [ ]:
import matplotlib.pyplot as plt
import cv2
import keras
import numpy as np
import os
import seaborn as sns
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix

Exploration des données


In [ ]:
from os import path
path1 = '/content/data/train/benign/100.jpg'
path2 = '/content/data/train/malignant/10.jpg'
img1 = cv2.imread(path1)
img2 = cv2.imread(path2)

In [ ]:
plt.imshow(img1)
plt.title("Image benign")


In [ ]:
plt.imshow(img2)
plt.title("Image malign")

In [ ]:
data = "/content/data"
#data = len(os.listdir(data))


In [ ]:
benign_train = "/content/data/train/benign"
malignant_train = "/content/data/train/malignant"
benign_test = "/content/data/test/benign"
malignant_test = "/content/data/test/malignant"

In [ ]:
nb_benign_train = len(os.listdir(benign_train))
nb_malignant_train = len(os.listdir(malignant_train))
nb_benign_test = len(os.listdir(benign_test))
nb_malignant_test = len(os.listdir(malignant_test))

In [ ]:
classes = ['Benignes', 'Malignes']
effectifs = [nb_benign_train, nb_malignant_train]

plt.bar(classes, effectifs, color=['blue', 'red'])
plt.xlabel('Classes')
plt.ylabel('Nombre d\'images')
plt.title('Distribution des classes dans l\'ensemble d\'entraînement')
plt.show()


In [ ]:
classes = ['Benignes', 'Malignes']
effectifs = [nb_benign_test, nb_malignant_test]

plt.bar(classes, effectifs, color=['blue', 'red'])
plt.xlabel('Classes')
plt.ylabel('Nombre d\'images')
plt.title('Distribution des classes dans l\'ensemble d\'entraînement')
plt.show()


In [ ]:
# Load in training pictures

X_benign_train = [np.array(Image.open(os.path.join(benign_train, filename)), dtype='uint8') for filename in os.listdir(benign_train)]

X_malign_train = [np.array(Image.open(os.path.join(malignant_train, filename)), dtype='uint8') for filename in os.listdir(malignant_train)]



In [ ]:
# Load in test pictures

X_benign_test = [np.array(Image.open(os.path.join(benign_test, filename)), dtype='uint8') for filename in os.listdir(benign_test)]

X_malign_test = [np.array(Image.open(os.path.join(malignant_test, filename)), dtype='uint8') for filename in os.listdir(malignant_test)]



In [ ]:
#label
y_benign_train = np.zeros(len(X_benign_train))
y_malign_train = np.ones(len(X_malign_train))

In [ ]:
y_benign_test = np.zeros(len(X_benign_test))
y_malign_test = np.ones(len(X_malign_test))

In [ ]:
# Concaténer les données d'entraînement et de test
X_train = np.concatenate([X_benign_train, X_malign_train], axis=0)
y_train = np.concatenate([y_benign_train, y_malign_train], axis=0)
X_test = np.concatenate([X_benign_test, X_malign_test], axis=0)
y_test = np.concatenate([y_benign_test, y_malign_test], axis=0)

In [ ]:


# Initialiser le modèle
model = Sequential()

# Première couche de convolution avec normalisation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Deuxième couche de convolution avec normalisation
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Troisième couche de convolution avec normalisation
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Couche d'aplatissement pour convertir la sortie en un vecteur
model.add(Flatten())

# Couches entièrement connectées
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))  # Couche de sortie pour la classification binaire

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Afficher la structure du modèle
model.summary()


In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


In [ ]:
# Prédire les classes sur l'ensemble de test
from sklearn.metrics import classification_report, confusion_matrix
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Afficher le rapport de classification et la matrice de confusion
print(classification_report(y_test, y_pred_classes))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
import seaborn as sns


# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Afficher la matrice de confusion avec seaborn (ou matplotlib)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.show()


In [ ]:
# prédire  avec de nouvelle données
new_image = np.array(Image.open('/content/dummy/30.jpg'), dtype='uint8')
new_image = new_image.reshape((1, *new_image.shape))

# Prédire la classe de la nouvelle image
prediction = model.predict(new_image)
if prediction > 0.5:
    print("La tumeur est maligne.")
else:
    print("La tumeur est bénigne.")


In [ ]:
Path = '/content/dummy/30.jpg'
new_image_30 = cv2.imread(Path)
plt.imshow(new_image_30)